# Create meta_file that contains path to rds files

In [1]:
library(data.table)
library(tidyverse)

Warning message:
"package 'purrr' was built under R version 4.4.2"
-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.0.4     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::between()     masks data.table::between()
x dplyr::filter()      masks stats::filter()
x dplyr::first()       masks data.table::first()
x lubridate::hour()    masks data.table::hour()
x lubridate::isoweek() masks data.table::isoweek()
x dplyr::lag()         masks stats::lag()
x dplyr::last()        masks data.table::last()
x lubridate::mday()    masks data.table::mday()
x lubridate::minute()  masks data.table::minute()
x lubridate::month()   masks data.table::month()
x lubridate::quarter() masks data.table::quarter()
x lubridate::second()  masks data.table::second()
x purr

In [2]:
all_gwas = fread("~/data/GWAS/AD_GWAS/ADGWAS.union_export.tsv.gz")

In [3]:
# Define the list of studies
studies = c(
  "AD_Bellenguez_2022",
  "AD_Bellenguez_EADB_2022",
  "AD_Bellenguez_EADI_2022",
  # "AD_Bellenguez_GRACE_2022",
  "AD_Jansen_2021",
  "AD_Kunkle_Stage1_2019",
  "AD_Wightman_Excluding23andMe_2021",
  "AD_Wightman_ExcludingUKBand23andME_2021",
  "AD_Wightman_Full_2021"
)

In [4]:
AD_data = all_gwas |> filter(study %in% studies, coverage == "top_loci_95")

In [5]:
AD_data = AD_data |> mutate(block = gsub("-", "_", block),
                            study_block = paste0(study,":",block))

In [6]:
tail(AD_data)

variants,cs_single_effect_regression,cs_noqc,cs_qc_impute,cs_qc_only,cs_conditional_regression_noqc,cs_conditional_regression_qc_impute,cs_conditional_regression_qc_only,coverage,pip_single_effect_regression,pip_noqc,pip_qc_impute,pip_qc_only,pip_conditional_regression_noqc,pip_conditional_regression_qc_impute,pip_conditional_regression_qc_only,study,block,study_block
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
8:94983473:C:T,1,0,1,0,1,1,1,top_loci_95,0.08845241,0.045955702,0.046885837,0.047240630,0.08899867,0.061267407,0.08899867,AD_Bellenguez_2022,8_93290173_95220317,AD_Bellenguez_2022_8_93290173_95220317
8:94987509:C:CA,0,0,1,0,0,1,0,top_loci_95,NA,NA,0.121038250,NA,NA,0.157460697,NA,AD_Bellenguez_2022,8_93290173_95220317,AD_Bellenguez_2022_8_93290173_95220317
8:95008746:T:C,1,0,1,0,1,1,1,top_loci_95,0.01393901,0.008267977,0.008250387,0.009323420,0.01443862,0.009941807,0.01443862,AD_Bellenguez_2022,8_93290173_95220317,AD_Bellenguez_2022_8_93290173_95220317
8:95026024:C:G,1,0,1,0,1,1,1,top_loci_95,0.01821405,0.010695903,0.010696676,0.011890104,0.01874527,0.012900527,0.01874527,AD_Bellenguez_2022,8_93290173_95220317,AD_Bellenguez_2022_8_93290173_95220317
8:95038329:T:C,1,0,1,0,1,1,1,top_loci_95,0.01304201,0.007791333,0.007760397,0.008854551,0.01354367,0.009325984,0.01354367,AD_Bellenguez_2022,8_93290173_95220317,AD_Bellenguez_2022_8_93290173_95220317
8:95041772:C:T,0,0,1,0,0,1,0,top_loci_95,NA,NA,0.033104743,NA,NA,0.045904424,NA,AD_Bellenguez_2022,8_93290173_95220317,AD_Bellenguez_2022_8_93290173_95220317


In [7]:
single_data = AD_data |> filter(cs_single_effect_regression != 0)
fine_map_data = AD_data |> filter(cs_qc_impute != 0)

In [8]:
single_only = unique(setdiff(single_data$study_block, fine_map_data$study_block))
finemap_only = unique(setdiff(fine_map_data$study_block, single_data$study_block))
both = unique(intersect(single_data$study_block, fine_map_data$study_block))

In [9]:


# Function to split study_block into study and block
split_study_block = function(df) {
  df %>%
    mutate(
      study = str_extract(study_block, "^[^:]+"),  # Extract part before ':'
      block = str_extract(study_block, "(?<=:).*"), # Extract part after ':'
      chrom = str_extract(block, "^[^_]+")
    )
}

# Create data for each category
# Create dataframes and apply split_study_block
df_single_only = data.frame(
  study_block = single_only) %>% split_study_block() %>%
  mutate(
    single_path = paste0("analysis_result/AD_GWAS_finemapping/NoQC_single_effect/", block, ".noQC.univariate_single_effect.rds"),
    finemap_path = NA
  )

df_finemap_only = data.frame(
  study_block = finemap_only) %>% split_study_block() %>%
  mutate(
    single_path = NA,
    finemap_path = paste0("GWAS/finemapping/2024_04/RSS_QC_RAISS_imputed.chr", block, ".univariate_susie_rss.rds")
  )

df_both = data.frame(
  study_block = both) %>% split_study_block() %>%
  mutate(
    single_path = paste0("analysis_result/AD_GWAS_finemapping/NoQC_single_effect/", block, ".noQC.univariate_single_effect.rds"),
    finemap_path = paste0("GWAS/finemapping/2024_04/RSS_QC_RAISS_imputed.chr", block, ".univariate_susie_rss.rds")
  )

# Combine all dataframes
meta_file_cloud = bind_rows(df_single_only, df_finemap_only, df_both) |> select(-study_block)
meta_file_cloud$single_path <- sapply(meta_file_cloud$single_path, function(path) {
  # Extract the basename
  basename <- basename(path)
  
  # Replace the second underscore with a dash in the basename
  new_basename <- sub("^([^_]+_[^_]+)_", "\\1-", basename)
  
  # Replace the old basename with the new one in the full path
  sub(basename, new_basename, path)
})
fwrite(meta_file_cloud, "meta_file_cloud.tsv", sep = '\t')

In [13]:
tail(meta_file_cloud)

,study,block,chrom,single_path,finemap_path
,<chr>,<chr>,<chr>,<chr>,<chr>
234,AD_Jansen_2021,8_27515963_29469590,8,analysis_result/AD_GWAS_finemapping/NoQC_single_effect/8_27515963-29469590.noQC.univariate_single_effect.rds,GWAS/finemapping/2024_04/RSS_QC_RAISS_imputed.chr8_27515963_29469590.univariate_susie_rss.rds
235,AD_Kunkle_Stage1_2019,8_27515963_29469590,8,analysis_result/AD_GWAS_finemapping/NoQC_single_effect/8_27515963-29469590.noQC.univariate_single_effect.rds,GWAS/finemapping/2024_04/RSS_QC_RAISS_imputed.chr8_27515963_29469590.univariate_susie_rss.rds
236,AD_Wightman_Full_2021,8_27515963_29469590,8,analysis_result/AD_GWAS_finemapping/NoQC_single_effect/8_27515963-29469590.noQC.univariate_single_effect.rds,GWAS/finemapping/2024_04/RSS_QC_RAISS_imputed.chr8_27515963_29469590.univariate_susie_rss.rds
237,AD_Wightman_Excluding23andMe_2021,8_27515963_29469590,8,analysis_result/AD_GWAS_finemapping/NoQC_single_effect/8_27515963-29469590.noQC.univariate_single_effect.rds,GWAS/finemapping/2024_04/RSS_QC_RAISS_imputed.chr8_27515963_29469590.univariate_susie_rss.rds
238,AD_Wightman_ExcludingUKBand23andME_2021,8_27515963_29469590,8,analysis_result/AD_GWAS_finemapping/NoQC_single_effect/8_27515963-29469590.noQC.univariate_single_effect.rds,GWAS/finemapping/2024_04/RSS_QC_RAISS_imputed.chr8_27515963_29469590.univariate_susie_rss.rds
239,AD_Bellenguez_2022,8_93290173_95220317,8,analysis_result/AD_GWAS_finemapping/NoQC_single_effect/8_93290173-95220317.noQC.univariate_single_effect.rds,GWAS/finemapping/2024_04/RSS_QC_RAISS_imputed.chr8_93290173_95220317.univariate_susie_rss.rds


sos run ../project/image_QTL/compare_single_finemap.ipynb susie_plot --meta-file ~/project/image_QTL/finemap_single.tsv --cwd ~/project/image_QTL/plots

In [ ]:
[global]
parameter: meta_file = path('meta_file_cloud.tsv') # path that contains different finemapping results
parameter: cwd = path()

import pandas as pd
# meta_file = pd.read_csv(meta_file,sep ="\t")
chrom = list(set(pd.read_csv(meta_file,sep ="\t")["chrom"]))
# chrom = list(set(meta_file["chrom"]))
# block = list(set(pd.read_csv(meta_file,sep ="\t")["block"]))
# study = list(set(pd.read_csv(meta_file,sep ="\t")["study"]))


In [ ]:
[susie_plot]
input: meta_file, for_each = "chrom"
output: f'{cwd}/{_input:bn}.chr{_chrom}.pdf'

R:expand = "${ }", stdout = f"{_output[0]:n}.stdout", stderr = f"{_output[0]:n}.stderr"
    library(data.table)
    library(susieR)

    # Function to create plot or placeholder
    susie_plot_wrapper = function(res, plot_type = "PIP", title = "No Plot") {
      if (is.null(res)) {
        # Create empty plot
        plot.new()
        title(main = title)
      } else {
        # Create plot
        susie_plot(res, plot_type, main = title)
      }
    }

    # Functions to get susie_result
    get_res = function(con_data) {
        if (length(con_data) == 0) return(NULL)
        
        res_name <- names(con_data)[1]
        if (length(con_data[[res_name]]) == 0 || 
            length(con_data[[res_name]]$susie_result_trimmed) == 0) {
            return(NULL)
            print(paste("susie_result is null for",con_data))
        }
        
        return(con_data[[res_name]]$susie_result_trimmed)
    }

    tryCatch({
        meta_file = fread("${meta_file}", sep = "\t")
        if (nrow(meta_file) == 0) {
            stop("Empty meta file")
        }
        # Filter for current chromosome
        meta_file = meta_file[chrom == ${_chrom}]
        message(sprintf("Processing chromosome %s with %d rows", ${_chrom}, nrow(meta_file)))
    }, error = function(e) {
        stop(paste("Failed to read meta file:", e$message))
    })
    # Calculate dimensions for the plot layout
    blocks = unique(meta_file$block)
    studies = unique(meta_file$study)
    n_blocks = length(blocks)
    n_studies = length(studies)

    pdf(file = "${_output}", height = 10 * n_blocks, width = 5 * n_studies, compress = TRUE)

    # Create a layout matrix where each row represents a study
    # and each row pair represents a block's finemap and single plots
    layout_matrix = matrix(0, nrow = 2 * n_studies, ncol = n_blocks)
    plot_counter = 1
    
    # Initialize layout matrix
    for (j in 1:n_studies) {
        for (i in 1:n_blocks) {
            layout_matrix[2*j-1, i] = plot_counter
            layout_matrix[2*j, i] = plot_counter + 1
            plot_counter = plot_counter + 2
        }
    }
    layout(layout_matrix)

    # Set plot parameters
    par(mar = c(2,2,2,2))
    par(pty = "s")
    # par(oma = c(0,0,0,0))
    # par(pin = c(3,3))
    # par(ps = 10)
    # par(cex = 1)
    # par(cex.axis = 1)
    # par(cex.lab = 1)
    # par(cex.main = 1.2)
    print(blocks)
    print(studies)
    # Create plots block by block
    for (b in blocks) {
        for (s in studies) {
            # Get the row for current block and study if it exists
            row = meta_file[which(meta_file$block == b & meta_file$study == s),]
            print("nrow")
            print(nrow(row))

            if (nrow(row) == 0) {
                # If no data for this block-study combination, create empty plots
                plot.new()
                title(main = paste("No Finemap Data\n", b, ":", s))
                plot.new()
                title(main = paste("No Single Data\n", b, ":", s))
                next
            }

            # Check and read single effect file
          single_data <- NULL
          if (!is.na(row$single_path) && file.exists(row$single_path)) {
              tryCatch({
                single_data = readRDS(row$single_path)
                # single_data = extract_layer(single_data, studies)
              }, error = function(e) {
                message(sprintf("Error reading single effect file for %s in %s: %s", 
                                b, s, e$message))
              })
          } else {
              message(sprintf("Single effect file path is NA or does not exist for %s in %s", row$block, s))
          }
            
          # Check and read finemapping file
          finemap_data <- NULL
          if (!is.na(row$finemap_path) && file.exists(row$finemap_path)) {
              tryCatch({
                finemap_data = readRDS(row$finemap_path)
                # finemap_data = extract_layer(finemap_data, studies)
              }, error = function(e) {
                message(sprintf("Error reading finemapping file for %s in %s: %s", 
                                b, s, e$message))
              })
          } else {
              message(sprintf("Finemapping file path is NA or does not exist for %s in %s", 
                              b, s))
          }
          
          
          
          # Create plots organized by block and study
          finemap_con_data = finemap_data[[s]]
          print(str(finemap_con_data))
          finemap_res = get_res(finemap_con_data)
          # Create PIP plot
          susie_plot_wrapper(finemap_res, "PIP", paste("Finemap PIP:", s, ":", b))
          # Get corresponding results
          single_con_data = single_data[[s]]
          print(str(single_con_data))
          single_res = get_res(single_con_data)
          # Create PIP plot
          susie_plot_wrapper(single_res, "PIP", paste("Single PIP:", s, ":", b))
        }
    }
    dev.off()